In [1]:
from DataProcessing import read_data, make_graphs
from models import GNN
from training import test, load_checkpoint
import torch
from torch_geometric.loader import DataLoader

Load model

In [5]:
model = GNN(3, 7, 3)
lossFunc = torch.nn.L1Loss(reduction='sum')

checkpoint_path = 'checkpoints/big_decoder_epoch_24.pth'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

C:\Users\cirob\AppData\Local\Temp\ipykernel_7348\2470577699.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


RuntimeError: Error(s) in loading state_dict for GNN:
	Unexpected key(s) in state_dict: "message_passing_layers.6.weight", "message_passing_layers.6.bias", "message_passing_layers.6.running_mean", "message_passing_layers.6.running_var", "message_passing_layers.6.num_batches_tracked", "message_passing_layers.7.mlp.mlp.0.weight", "message_passing_layers.7.mlp.mlp.0.bias", "message_passing_layers.7.mlp.mlp.3.weight", "message_passing_layers.7.mlp.mlp.3.bias", "message_passing_layers.7.mlp.mlp.6.weight", "message_passing_layers.7.mlp.mlp.6.bias", "message_passing_layers.7.mlp.mlp.9.weight", "message_passing_layers.7.mlp.mlp.9.bias", "message_passing_layers.7.mlp.mlp.11.weight", "message_passing_layers.7.mlp.mlp.11.bias". 

Test large water system

In [2]:
charges = [None, -0.82, 0.41]
LJ_params = [None, (0.155, 3.165), (0, 0)]
data = read_data(['data/big_water.lammpstrj'])
test_graphs = make_graphs(data, charges, LJ_params, cutoff=2.3)
test_loader = DataLoader(test_graphs, batch_size=32)

test_loss = test(model, test_loader, lossFunc)
test_loss

KeyboardInterrupt: 

Test argon system

In [3]:
charges = [None, 0]
LJ_params = [None, (0.2378, 3.405)]
data = read_data(['data/argon.lammpstrj'], molecular=False)
test_graphs = make_graphs(data, charges, LJ_params, cutoff=3.5)
test_loader = DataLoader(test_graphs, batch_size=32)

test_loss = test(model, test_loader, lossFunc)
test_loss

NameError: name 'model' is not defined

In [5]:
test_graphs

[Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[0, 7], y=[400, 3]),
 Data(x=[400, 3], edge_index=[2, 0], edge_attr=[